In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

2022-04-14 12:31:38.314841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-14 12:31:38.315095: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print("loading images")
data = []
labels = []

imagePaths = sorted(list(paths.list_images("Dataset")))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

loading images


In [3]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [4]:
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)


In [5]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(len(lb.classes_), activation="sigmoid"))

2022-04-14 12:31:54.211100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-14 12:31:54.211145: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-14 12:31:54.211174: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (awi-stavya): /proc/driver/nvidia/version does not exist
2022-04-14 12:31:54.211462: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
INIT_LR = 0.01
EPOCHS = 15

print("training network")
opt = SGD(lr=INIT_LR)

model.compile(loss="categorical_crossentropy", optimizer=opt,
            metrics=["accuracy"])

H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=15)

training network


/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2022-04-14 12:31:54.624437: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18186240 exceeds 10% of free system memory.


Epoch 1/15
99/99 [==============================] - 6s 51ms/step - loss: 0.7778 - accuracy: 0.6149 - val_loss: 0.6979 - val_accuracy: 0.6619
Epoch 2/15
99/99 [==============================] - 4s 41ms/step - loss: 0.6640 - accuracy: 0.6736 - val_loss: 0.7736 - val_accuracy: 0.5364
Epoch 3/15
99/99 [==============================] - 4s 39ms/step - loss: 0.6308 - accuracy: 0.7027 - val_loss: 0.7043 - val_accuracy: 0.5911
Epoch 4/15
99/99 [==============================] - 4s 39ms/step - loss: 0.6204 - accuracy: 0.7014 - val_loss: 0.6083 - val_accuracy: 0.7004
Epoch 5/15
99/99 [==============================] - 4s 43ms/step - loss: 0.5908 - accuracy: 0.7284 - val_loss: 0.5902 - val_accuracy: 0.7126
Epoch 6/15
99/99 [==============================] - 4s 42ms/step - loss: 0.5722 - accuracy: 0.7338 - val_loss: 0.5839 - val_accuracy: 0.7146
Epoch 7/15
99/99 [==============================] - 5s 50ms/step - loss: 0.5467 - accuracy: 0.7439 - val_loss: 0.9389 - val_accuracy: 0.5202
Epoch 8/15
99

In [9]:
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=32)

print(predictions)

[INFO] evaluating network...
[[0.9131098  0.43588918 0.09295484]
 [0.8426304  0.69809026 0.05021968]
 [0.7927152  0.89928305 0.03319663]
 ...
 [0.74331534 0.9471692  0.02344072]
 [0.89985526 0.6368341  0.04155031]
 [0.84781885 0.30185968 0.20537248]]


In [8]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.65      0.89      0.75       233
single_person       0.85      0.60      0.71       249
triple_person       0.00      0.00      0.00        12

     accuracy                           0.72       494
    macro avg       0.50      0.50      0.49       494
 weighted avg       0.74      0.72      0.71       494



/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,